In [ ]:
import requests
import json
import pandas as pd
import pickle

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
# Defining api-endpoint
ENDPOINT = "https://recruitment.aimtechnologies.co/ai-tasks"
DATASET_PATH = '/gdrive/MyDrive/dialect_dataset.csv'

In [ ]:
dialect_data = pd.read_csv(DATASET_PATH)
dialect_data['id'] = dialect_data['id'].astype(str)
id_retrieval_list = list(dialect_data['id'])

In [ ]:
class DataFetch:
  session = None
  data = []
  def __init__(self, url, req_capacity):
    self.endpoint = url
    self.capacity = req_capacity
  
  # Initialize session for multiple requests
  def initialize_connection(self):
    self.session = requests.Session()

  # Function to retrieve text data by id
  def retrieve_data(self, request_body):
    total_request_length = len(request_body)
    for i in range(0, len(request_body), self.capacity):
      capped_request = json.dumps(request_body[i : i + self.capacity])
      req = requests.post(url=self.endpoint, data=capped_request)
      capped_data = list(req.json().values())
      self.data.extend(capped_data)
    return self.data 
  
  # Close session 
  def close_connection(self): 
    self.session.close()
    self.session = None

In [ ]:
fetcher = DataFetch(ENDPOINT, 1000)
fetcher.initialize_connection()

In [ ]:
text = fetcher.retrieve_data(id_retrieval_list)

In [ ]:
len(id_retrieval_list) == len(text)

True

In [ ]:
fetcher.close_connection()

In [ ]:
# Add text to csv
dialect_data['text'] = text

In [ ]:
# Save new dataset
with open('/gdrive/MyDrive/text_dialect_dataset.pkl', 'wb') as f:
  pickle.dump(dialect_data, f)